In [1]:
import pandas as pd
import numpy as np

In [ ]:
# Задание 1
# Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте
# среднее время жизни пользователей, которые выставили более 100 оценок.
# Под временем жизни понимается разница между максимальным и минимальным
# значением столбца timestamp для данного значения userId.

ratings = pd.read_csv('ml-latest-small/ratings.csv')

# np.ptp - Range of values (maximum - minimum) along an axis.
live_avg = ratings.groupby('userId').agg(
    user_live=('timestamp', np.ptp),
    rating_count=('rating', 'count')
).query('rating_count > 100')['user_live'].mean()

In [2]:
# Задание 2
# Дана статистика услуг перевозок клиентов компании по типам (см. файл
# с кодом занятия). Необходимо сформировать две таблицы:
#   * таблицу с тремя типами выручки для каждого client_id без указания
#     адреса клиента
#   * аналогичную таблицу по типам выручки с указанием адреса клиента
#
# Обратите внимание, что в процессе объединения таблиц данные
# не должны теряться.

# - rzd - железнодорожные перевозки
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

# - auto - автомобильные перевозки
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

# - air - воздушные перевозки
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

# - client_base - адреса клиентов
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а',
                    'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [4]:
table = rzd.merge(auto, how='outer', on='client_id')
table = table.merge(air, how='outer', on='client_id').fillna(0)

# >>> table
#    client_id  rzd_revenue  auto_revenue  air_revenue
# 0        111       1093.0           0.0          0.0
# 1        112       2810.0           0.0          0.0
# 2        113      10283.0       57483.0          0.0
# 3        114       5774.0          83.0          0.0
# 4        115        981.0         912.0         81.0
# 5        116          0.0        4834.0          4.0
# 6        117          0.0          98.0         13.0
# 7        118          0.0           0.0        173.0

full_table = table.merge(client_base, how='outer', on='client_id')

# >>> full_table
#    client_id  rzd_revenue  auto_revenue  air_revenue          address
# 0        111       1093.0           0.0          0.0  Комсомольская 4
# 1        112       2810.0           0.0          0.0   Энтузиастов 8а
# 2        113      10283.0       57483.0          0.0  Левобережная 1а
# 3        114       5774.0          83.0          0.0          Мира 14
# 4        115        981.0         912.0         81.0        ЗЖБИиДК 1
# 5        116          0.0        4834.0          4.0    Строителей 18
# 6        117          0.0          98.0         13.0  Панфиловская 33
# 7        118          0.0           0.0        173.0     Мастеркова 4

In [ ]:
# Задание 3
# В задаче сквозной аналитики вам предоставили данные по местоположению
# пользователей. Т. е. для каждого user_id известна последовательность
# координат (широта/долгота), когда они требовались приложению для полноценной
# работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?
# Для составления ответа можно использовать вопросы:
#   * У каждого пользователя известен набор координат. А для связывания
#     с визитом или фактом покупки скорее всего потребуется одно-два числа.
#     Как их получить?
#   * Наборы координат одного и того же пользователя могут быть значительно
#     удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
#   * Какие дополнительные признаки можно получить из координат? Ведь это
#     просто числа, которые сами по себе мало что дают.

# Решение
# Для того что бы добавить координаты нужно ввести индификатор визита
# так как пользователь может зайти из разных мест

coordinates = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'visit_id': [116, 234, 643, 34, 67],
        'latitude': [-34.58, -15.78, -33.45, 4.60, 10.48],
        'longitude': [-58.66, -47.91, -70.66, -74.08, -66.86]
    }
)

# Из координат можно получить данные откуда делаются больше всего запросов,
# что в этой области предпочитают и т. д. Бизнесу будет значительно проще
# повысить рентабельность инвестиций в рекламу, каналы продаж.